In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE 


In [2]:
dataset=pd.read_csv('prep.csv')

In [3]:
datas=pd.get_dummies(dataset,drop_first=True)

In [4]:
datas

age         bp   al   su         bgr          bu        sc  \
0     2.000000  76.459948  3.0  0.0  148.112676   57.482105  3.077356   
1     3.000000  76.459948  2.0  0.0  148.112676   22.000000  0.700000   
2     4.000000  76.459948  1.0  0.0   99.000000   23.000000  0.600000   
3     5.000000  76.459948  1.0  0.0  148.112676   16.000000  0.700000   
4     5.000000  50.000000  0.0  0.0  148.112676   25.000000  0.600000   
..         ...        ...  ...  ...         ...         ...       ...   
394  51.492308  70.000000  0.0  0.0  219.000000   36.000000  1.300000   
395  51.492308  70.000000  0.0  2.0  220.000000   68.000000  2.800000   
396  51.492308  70.000000  3.0  0.0  110.000000  115.000000  6.000000   
397  51.492308  90.000000  0.0  0.0  207.000000   80.000000  6.800000   
398  51.492308  80.000000  0.0  0.0  100.000000   49.000000  1.000000   

            sod       pot       hrmo  ...  pc_normal  pcc_present  ba_present  \
0    137.528754  4.627244  12.518156  ...          0            0           0   
1    137.528754  4.627244  10.700000  ...          1            0           0   
2    138.000000  4.400000  12.000000  ...          1            0           0   
3    138.000000  3.200000   8.100000  ...          1            0           0   
4    137.528754  4.627244  11.800000  ...          1            0           0   
..          ...       ...        ...  ...        ...          ...         ...   
394  139.000000  3.700000  12.500000  ...          1            0           0   
395  137.528754  4.627244   8.700000  ...          1            0           0   
396  134.000000  2.700000   9.100000  ...          1            0           0   
397  142.000000  5.500000   8.500000  ...          1            0           0   
398  140.000000  5.000000  16.300000  ...          1            0           0   

     htn_yes  dm_yes  cad_yes  appet_yes  pe_yes  ane_yes  classification_yes  
0          0       0        0          1       1        0                   1  
1          0       0        0          1       0        0                   1  
2          0       0        0          1       0        0                   1  
3          0       0        0          1       0        1                   1  
4          0       0        0          1       0        0                   1  
..       ...     ...      ...        ...     ...      ...                 ...  
394        0       0        0          1       0        0                   1  
395        1       1        0          1       0        1                   1  
396        1       1        0          0       0        0                   1  
397        1       1        0          1       0        1                   1  
398        0       0        0          1       0        0                   0  

[399 rows x 28 columns]

In [5]:
independent=datas.drop("classification_yes",1)
dependent=datas['classification_yes']

In [6]:
def split_scaler(independent,dependent):
    X_train,X_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.25,random_state=0)
    sc=StandardScaler()
    X_train=sc.fit_transform(X_train)
    X_test=sc.transform(X_test)
    return X_train,X_test,y_train,y_test

In [7]:
def cm_pred(classifier,X_test):
    y_pred=classifier.predict(X_test)
    from sklearn.metrics import confusion_matrix
    cm=confusion_matrix(y_test,y_pred)
    from sklearn.metrics import classification_report
    clf=classification_report(y_test,y_pred)
    from sklearn.metrics import accuracy_score 
    accurancy=accuracy_score(y_test,y_pred)
    return classifier,cm,clf,accurancy,X_test,y_test

In [8]:
def logistic(X_train,y_train,X_test):
    from sklearn.linear_model import LogisticRegression
    classifier=LogisticRegression(random_state = 0)
    classifier.fit(X_train,y_train)
    classifier,cm,clf,accurancy,X_test,y_test=cm_pred(classifier,X_test)
    return classifier,cm,clf,accurancy,X_test,y_test
    

In [9]:
def random(X_train,y_train,X_test):
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train,y_train)
    classifier,cm,clf,accurancy,X_test,y_test=cm_pred(classifier,X_test)
    #return classifier,cm,clf,accurancy,X_test,y_test
    return classifier,cm,clf,accurancy,X_test,y_test

In [10]:
def decision(X_train,y_train,X_test):
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
    classifier.fit(X_train,y_train)
    classifier,cm,clf,accurancy,X_test,y_test=cm_pred(classifier,X_test)
    return classifier,cm,clf,accurancy,X_test,y_test

In [11]:
def select_feature(independent,dependent,n):
    rfelist=[]
    from sklearn.linear_model import LogisticRegression
    logis=LogisticRegression(solver='lbfgs')
    from sklearn.ensemble import RandomForestClassifier
    #from sklearn.ensemble import RandomForestClassifier
    ran=RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
    from sklearn.tree import DecisionTreeClassifier
    deci=DecisionTreeClassifier(criterion = 'gini', max_features='sqrt',splitter='best',random_state = 0)
    list1=[logis,ran,deci]
    for i in list1:
        test=RFE(i,n)
        test1=test.fit(independent,dependent)
        feature_sele=test1.transform(independent)
        rfelist.append(feature_sele)
    return rfelist

In [17]:
def dataframe(acclogis,accran,accdeci):
    table=pd.DataFrame(index=['logistic','random','decision'],columns=['logistic','random','decision'])
    for number,indx in enumerate (table.index):
        table['logistic'][indx]=acclogis[number]
        table['random'][indx]=accran[number]
        table['decision'][indx]=accdeci[number]
        
        
    return table

In [18]:
acclogis=[]
accran=[]
accdeci=[]


In [19]:
rfelist=select_feature(independent,dependent,5)

C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes a

C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: dis

In [20]:
for i in rfelist:
     X_train, X_test, y_train, y_test=split_scaler(independent,dependent)
     classifier,cm,clf,accurancy,X_test,y_test=logistic(X_train,y_train,X_test)
     acclogis.append(accurancy)
     classifier,cm,clf,accurancy,X_test,y_test=random(X_train,y_train,X_test)
     accran.append(accurancy)
     classifier,cm,clf,accurancy,X_test,y_test=decision(X_train,y_train,X_test)
     accdeci.append(accurancy)


C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Priya\Anaconda3\envs\aiml\lib\site-packages\sklearn\utils\fixes.py:230: DeprecationWarning: distutils Version c

In [21]:
result=dataframe(acclogis,accran,accdeci)

In [22]:
result

logistic random decision
logistic     0.99   0.99      0.9
random       0.99   0.99      0.9
decision     0.99   0.99      0.9